In [1]:
import pandas as pd 
import numpy as np 
import random

In [34]:
df = pd.read_csv("Concatenated_Job_Description.csv")
df

,RequisitionID,CleanedJobTitle,JobDescription,EditedJobDescription
0,143642,Information Security EngineerLevel Cloud,Requisition Number 143642 Job Description Cint...,"{\n ""marketing"": [\n ""Cintas offers compre..."
1,SENIO004432,Senior Engineer Electronics Commercial Develop...,Description Purpose Position Develop from conc...,"{\n ""marketing"": [\n ""Develop from concept..."
2,R-021199,Engineer DER Planning Hybrid,hybrid role . first three months fulltime offi...,"{\n ""marketing"": [\n ""largest Energy..."
3,356-383,Cybersecurity Engineer,position support systems configuration within ...,"{\n ""marketing"": [\n ""Support Cybersecurit..."
4,234532W,Fire Protection Engineer,DuPont working things matter whether providing...,"{\n ""marketing"": [\n ""Get know Purpose mak..."
...,...,...,...,...
495,01.01.16-2023-21,Computer Engineer,Dynetics wholly owned subsidiary Leidos seekin...,"{\n ""marketing"": [],\n ""description"": [\n ..."
496,334701,Multivendor Customer Service Engineer MIUS Phi...,Nows time inspire future healthcaretogether. A...,"{\n ""marketing"": [\n ""At Siemens Healthine..."
497,R2028871,Engineer,Location Four Houston Center 1200 McKinney Sui...,"{\n ""marketing"":[\n ""Location Four Hous..."
498,2023-30496,Engineer,Pyramid Global Hospitality people come first. ...,"{\n ""marketing"": [\n ""Pyramid Global Hospi..."


In [35]:
with open("processed_ids.txt", "r") as file:
    processed_ids = [line.strip() for line in file.readlines()]
df_ids = df["RequisitionID"].tolist()
are_equal = set(df_ids) == set(processed_ids)
are_equal

True

In [4]:
final_sentences = []
final_labels = []

# Modify the function to correctly label the sentences
def extract_and_correctly_label_sentences(sentences_dict):
    capture = False
    for label, sentence_list in sentences_dict.items():
        if label == 'description':
            capture = True
        elif label == 'requirements':
            capture = False

        for sentence in sentence_list:
            final_sentences.append(sentence)
            final_labels.append(1 if capture else 0)

In [5]:
import json

In [6]:
new_df = pd.read_csv("LabeledSentences.csv")
new_df

,Sentence,IsDescription
0,Cintas offers comprehensive competitive medica...,0
1,One medical plan options even offered zero cos...,0
2,Additionally employeepartners enjoy Competitiv...,0
3,Cintas currently looking Security EngineerLeve...,1
4,role researches recommends implements changes ...,1
...,...,...
22648,Systems troubleshooting Root Cause Analysis ex...,0
22649,Strong communication skills including managing...,0
22650,An Equal Opportunity Employer Abbot welcomes e...,0
22651,We provide reasonable accommodation qualified ...,0


In [8]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression

In [10]:
# Use Python's split() method as a basic tokenizer
basic_tokenizer = lambda sentence: sentence.lower().split()

# Tokenize the sentences using the basic tokenizer
tokenized_sentences = [basic_tokenizer(sentence) for sentence in new_df['Sentence']]

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Verify the model by checking word vectors for a sample word
sample_word = "job"
if sample_word in word2vec_model.wv:
    print(f"Word2Vec model trained. Vector for the word '{sample_word}':")
    print(word2vec_model.wv[sample_word])

Word2Vec model trained. Vector for the word 'job':
[-0.59838045  0.58202773 -0.3205814  -0.36117712 -0.6945367  -0.47145876
  0.754385    1.812066   -0.644244   -0.6961992  -0.79554206 -0.6299825
 -0.39248458 -0.23227027  0.29279447 -1.3041778   0.13627745 -1.006712
 -0.44125432 -0.95229024  0.63237214  0.16028443  0.6294503  -0.32398748
 -0.5417498  -0.05759653 -0.21070206 -1.3387562  -0.7790999  -0.10182466
 -0.09375796  0.3572365   0.11375499 -0.46400416  0.04471982  1.2566655
 -0.23290285  0.03836777 -0.6206779  -1.2062376   0.63879186 -0.58793586
  0.23651281 -0.6124057   1.0160844  -0.2762938  -0.34811327  0.683941
  0.90132606  0.6691865   0.77882415 -0.48822063 -0.48370022  0.19180506
 -0.49403372  0.6650506   0.04449012  0.05634573 -1.0196748  -0.2586927
  0.1314479  -0.08031622  0.07190294 -0.49204707 -0.22117673  0.7747504
 -0.14548974  0.64182603 -0.34435242  0.70454973  0.08042607  0.68201005
  1.4418957   0.00518806  0.48444322  1.0512934   0.6459932  -0.0780334
 -0.01029

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Function to get Word2Vec embedding for a sentence
def get_word2vec_embedding(sentence, model):
    words = basic_tokenizer(sentence)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Generate Word2Vec embeddings for each sentence
X = np.array([get_word2vec_embedding(sentence, word2vec_model) for sentence in new_df['Sentence']])
y = new_df['IsDescription'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the logistic regression classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Evaluate the model
score = clf.score(X_test, y_test)
print(f"Test Accuracy: {score * 100:.2f}%")

Test Accuracy: 82.15%


C:\Users\13479\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
original_df = pd.read_csv('New_Cleaned_Engineer.csv')
original_df.drop(original_df.index[0],inplace = True)
original_df.reset_index(drop = True, inplace = True)
original_df

,RequisitionID,CleanedJobTitle,JobDescription
0,00000224907,Guidance Navigation Control GNC Engineer Lead,The Boeing Company search Lead Guidance Naviga...
1,00000331804,Propulsion Engineer Associate MidLevel Experie...,Job Description At Boeing innovate collaborate...
2,00000336462,Senior Process Controls Engineer,Job Description At Boeing innovate collaborate...
3,00000338951,RFMicrowave Engineer Level,Job Description At Boeing innovate collaborate...
4,00000339390,Interior Stress Analysis Engineer Career Expert,Job Description At Boeing innovate collaborate...
...,...,...,...
20131,WYMP-23-12039375-MG,Mining Engineer,Summary Explore new career BLM people precious...
20132,Y7193L,STATIONARY ENGINEER HELPER,STATIONARY ENGINEER HELPER Print http .?jobID3...
20133,Y7198M,STATIONARY ENGINEER II,STATIONARY ENGINEER II Print http .?jobID29656...
20134,Y7200A,STATIONARY ENGINEER CONTROLS SPECIALIST,STATIONARY ENGINEER CONTROLS SPECIALIST Print ...


In [18]:
with open("processed_ids.txt", "r") as file:
    processed_ids = file.read().splitlines()
# Filter out the rows that have already been processed
remaining_df = original_df[~original_df['RequisitionID'].isin(processed_ids)]
remaining_df

,RequisitionID,CleanedJobTitle,JobDescription
0,00000224907,Guidance Navigation Control GNC Engineer Lead,The Boeing Company search Lead Guidance Naviga...
1,00000331804,Propulsion Engineer Associate MidLevel Experie...,Job Description At Boeing innovate collaborate...
2,00000336462,Senior Process Controls Engineer,Job Description At Boeing innovate collaborate...
3,00000338951,RFMicrowave Engineer Level,Job Description At Boeing innovate collaborate...
4,00000339390,Interior Stress Analysis Engineer Career Expert,Job Description At Boeing innovate collaborate...
...,...,...,...
20131,WYMP-23-12039375-MG,Mining Engineer,Summary Explore new career BLM people precious...
20132,Y7193L,STATIONARY ENGINEER HELPER,STATIONARY ENGINEER HELPER Print http .?jobID3...
20133,Y7198M,STATIONARY ENGINEER II,STATIONARY ENGINEER II Print http .?jobID29656...
20134,Y7200A,STATIONARY ENGINEER CONTROLS SPECIALIST,STATIONARY ENGINEER CONTROLS SPECIALIST Print ...


In [19]:
remaining_df.to_csv("Remaining.csv", index = False)

In [20]:
import json
from nltk.tokenize import sent_tokenize

# Initialize an empty list to store the labeled job descriptions
final_labeled_descriptions = []

# Loop through each row in the remaining DataFrame
for index, row in remaining_df.iterrows():
    job_description = row['JobDescription']
    sentences = sent_tokenize(job_description)
    
    description_sentences = []
    
    # Loop through each sentence
    for sentence in sentences:
        # Generate Word2Vec embedding for the sentence
        embedding = get_word2vec_embedding(sentence, word2vec_model)
        
        # Classify the sentence using the trained classifier
        label = clf.predict(embedding.reshape(1, -1))[0]
        
        # If the label is 'description', add the sentence to description_sentences
        if label == 1:
            description_sentences.append(sentence)
    
    # Convert the description_sentences list to a JSON string and append it to final_labeled_descriptions
    final_labeled_descriptions.append(json.dumps({"description": description_sentences}))

# Create the final DataFrame
final_df = pd.DataFrame({
    'RequisitionID': remaining_df['RequisitionID'],
    'CleanedJobTitle': remaining_df['CleanedJobTitle'],
    'EditedJobDescription': final_labeled_descriptions
})


In [21]:
final_df

,RequisitionID,CleanedJobTitle,EditedJobDescription
0,00000224907,Guidance Navigation Control GNC Engineer Lead,"{""description"": [""Selected candidates work wid..."
1,00000331804,Propulsion Engineer Associate MidLevel Experie...,"{""description"": [""core organization supports p..."
2,00000336462,Senior Process Controls Engineer,"{""description"": [""As Senior Process Controls E..."
3,00000338951,RFMicrowave Engineer Level,"{""description"": [""Performs trade studies model..."
4,00000339390,Interior Stress Analysis Engineer Career Expert,"{""description"": [""This position must meet Expo..."
...,...,...,...
20131,WYMP-23-12039375-MG,Mining Engineer,"{""description"": [""Evaluations reports models d..."
20132,Y7193L,STATIONARY ENGINEER HELPER,"{""description"": [""may reject application time ..."
20133,Y7198M,STATIONARY ENGINEER II,"{""description"": [""Keeps logs plant operations ..."
20134,Y7200A,STATIONARY ENGINEER CONTROLS SPECIALIST,"{""description"": [""Inspects operates monitors c..."


In [29]:
random_index = random.randint(0, len(final_df) - 1)
random_description_json = final_df.iloc[random_index]['EditedJobDescription']

# Parse the JSON string to a dictionary
random_description_dict = json.loads(random_description_json)

# Show the description sentences
random_description_dict.get('description', 'No description sentences found')

['Navarro Research Engineering recruiting Project Engineer Richland WA.',
 'This position support NavarroATL.',
 'The Facility Improvements Project Engineer supports crossfunctional group successfully execute variety facility improvement projects Hanford 222S Laboratory Complex.',
 'Projects must coordinated ongoing laboratory engineering analytical maintenance operations disrupt operations.',
 'This position direct report NavarroATL Laboratory Projects Director.',
 'Responsibilities Project Engineer Associate Under direct supervision Assist coordination multidiscipline teams support delivery projects modest size complexity.',
 'Assist development maintain control engineering elements project execution baseline including scope schedule budget.',
 'Proactively liaise engineering manager procurement ensure timely engagement support resources.',
 'Assist technical oversight engineered equipment procurement engineering services including things as technical procurement specifications state

In [36]:
df.drop(columns = 'JobDescription', inplace = True)
df

,RequisitionID,CleanedJobTitle,EditedJobDescription
0,143642,Information Security EngineerLevel Cloud,"{\n ""marketing"": [\n ""Cintas offers compre..."
1,SENIO004432,Senior Engineer Electronics Commercial Develop...,"{\n ""marketing"": [\n ""Develop from concept..."
2,R-021199,Engineer DER Planning Hybrid,"{\n ""marketing"": [\n ""largest Energy..."
3,356-383,Cybersecurity Engineer,"{\n ""marketing"": [\n ""Support Cybersecurit..."
4,234532W,Fire Protection Engineer,"{\n ""marketing"": [\n ""Get know Purpose mak..."
...,...,...,...
495,01.01.16-2023-21,Computer Engineer,"{\n ""marketing"": [],\n ""description"": [\n ..."
496,334701,Multivendor Customer Service Engineer MIUS Phi...,"{\n ""marketing"": [\n ""At Siemens Healthine..."
497,R2028871,Engineer,"{\n ""marketing"":[\n ""Location Four Hous..."
498,2023-30496,Engineer,"{\n ""marketing"": [\n ""Pyramid Global Hospi..."


In [38]:
# Initialize a list to store the filtered job descriptions
filtered_descriptions = []

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    try:
        # Parse the JSON in the 'EditedJobDescription' column
        job_desc = json.loads(row['EditedJobDescription'])
        
        # Initialize capture flag and list to store captured sentences
        capture = False
        captured_sentences = []
        
        # Loop through each label and its corresponding sentences
        for label, sentences in job_desc.items():
            if label == 'description' or label == 'Description':
                capture = True
            
            if capture:
                captured_sentences.extend(sentences)
                
            if label == 'requirements' or label == 'Requirements':
                capture = False
                break

        # Convert the list of captured sentences to a JSON string
        filtered_description = json.dumps({"description": captured_sentences})
        
        # Append the filtered description to the list
        filtered_descriptions.append(filtered_description)
        
    except json.JSONDecodeError:
        # Handle JSON decoding errors (if any)
        print(f"Error decoding JSON for index {index}")
        filtered_descriptions.append("")

# Add the filtered descriptions to a new column in the DataFrame
df['EditedJobDescription'] = filtered_descriptions

# Show the first few rows of the updated DataFrame
df


Error decoding JSON for index 59
Error decoding JSON for index 78
Error decoding JSON for index 129
Error decoding JSON for index 360
Error decoding JSON for index 468


,RequisitionID,CleanedJobTitle,EditedJobDescription
0,143642,Information Security EngineerLevel Cloud,"{""description"": [""Cintas currently looking Sec..."
1,SENIO004432,Senior Engineer Electronics Commercial Develop...,"{""description"": [""Principal Responsibilities L..."
2,R-021199,Engineer DER Planning Hybrid,"{""description"": [""The DER Planning Engineer Sy..."
3,356-383,Cybersecurity Engineer,"{""description"": [""position support systems con..."
4,234532W,Fire Protection Engineer,"{""description"": [""The Environmental Health Saf..."
...,...,...,...
495,01.01.16-2023-21,Computer Engineer,"{""description"": [""Dynetics wholly owned subsid..."
496,334701,Multivendor Customer Service Engineer MIUS Phi...,"{""description"": [""Siemens Healthineers continu..."
497,R2028871,Engineer,"{""description"": [""The EngineerOperator respons..."
498,2023-30496,Engineer,"{""description"": [""Pyramid Global Hospitality o..."


In [40]:
# Concatenate final_df and concatenated_df
final_combined_df = pd.concat([final_df, df], ignore_index=True)
final_combined_df

,RequisitionID,CleanedJobTitle,EditedJobDescription
0,00000224907,Guidance Navigation Control GNC Engineer Lead,"{""description"": [""Selected candidates work wid..."
1,00000331804,Propulsion Engineer Associate MidLevel Experie...,"{""description"": [""core organization supports p..."
2,00000336462,Senior Process Controls Engineer,"{""description"": [""As Senior Process Controls E..."
3,00000338951,RFMicrowave Engineer Level,"{""description"": [""Performs trade studies model..."
4,00000339390,Interior Stress Analysis Engineer Career Expert,"{""description"": [""This position must meet Expo..."
...,...,...,...
20131,01.01.16-2023-21,Computer Engineer,"{""description"": [""Dynetics wholly owned subsid..."
20132,334701,Multivendor Customer Service Engineer MIUS Phi...,"{""description"": [""Siemens Healthineers continu..."
20133,R2028871,Engineer,"{""description"": [""The EngineerOperator respons..."
20134,2023-30496,Engineer,"{""description"": [""Pyramid Global Hospitality o..."


In [51]:
# Filter out rows where 'EditedJobDescription' contains an empty "description" array
final_combined_df = final_combined_df[final_combined_df['EditedJobDescription'] != '{"description": []}']
final_combined_df

,RequisitionID,CleanedJobTitle,EditedJobDescription
0,00000224907,Guidance Navigation Control GNC Engineer Lead,"{""description"": [""Selected candidates work wid..."
1,00000331804,Propulsion Engineer Associate MidLevel Experie...,"{""description"": [""core organization supports p..."
2,00000336462,Senior Process Controls Engineer,"{""description"": [""As Senior Process Controls E..."
3,00000338951,RFMicrowave Engineer Level,"{""description"": [""Performs trade studies model..."
4,00000339390,Interior Stress Analysis Engineer Career Expert,"{""description"": [""This position must meet Expo..."
...,...,...,...
20131,01.01.16-2023-21,Computer Engineer,"{""description"": [""Dynetics wholly owned subsid..."
20132,334701,Multivendor Customer Service Engineer MIUS Phi...,"{""description"": [""Siemens Healthineers continu..."
20133,R2028871,Engineer,"{""description"": [""The EngineerOperator respons..."
20134,2023-30496,Engineer,"{""description"": [""Pyramid Global Hospitality o..."


In [54]:
random_index = random.randint(0, len(final_df) - 1)
random_description_json = final_combined_df.iloc[random_index]['EditedJobDescription']

# Parse the JSON string to a dictionary
random_description_dict = json.loads(random_description_json)

# Show the description sentences
random_description_dict.get('description', 'No description sentences found')

['senior RVDT engineer become familiar new product development process NPDP.',
 'Working part multidisciplinary team senior RVDT engineer responsibilities include extracting design information internal customer industry specifications order apply product guidelines part sizing performance predictions creating designs approving design changes.',
 'Role include new product development product improvements creating testing mockup hardware performance verification contributing creation new product guidelines.',
 'position requires close collaboration customer supplier operations manufacturing internal research development resources.',
 'Specific Duties New product design development Review summarize requirements customer internal industry specifications Interface suppliers customers internal cross functional teams Plans schedules coordinates detailed phases technical challenges Provide design guidance customers cross functional team Devises new approaches problems encountered.',
 'Particip

In [ ]:
final_combined_df.to_csv("Only_Description_Sentences.csv", index = False)